In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, mean, when

spark = SparkSession.builder.appName("Internal3").getOrCreate()

24/12/30 06:29:49 WARN Utils: Your hostname, arch resolves to a loopback address: 127.0.1.1; using 10.20.31.204 instead (on interface wlp5s0)
24/12/30 06:29:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 06:29:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/30 06:30:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
csv_path = "Datasets/Cleaned_DS_Jobs.csv"

In [6]:
jobs_df = spark.read.csv(csv_path, header = True, inferSchema = True)

In [7]:
df = jobs_df

In [8]:
df.show()

+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+
|           Job Title|Salary Estimate|Rating|         Location|                Size|   Type of ownership|            Industry|              Sector|job_state|company_age|python|spark|tableau|
+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+
|   Sr Data Scientist|       137-171 |   3.1|     New York, NY|1001 to 5000 empl...|Nonprofit Organiz...|  Insurance Carriers|           Insurance|       NY|         27|     0|    0|      0|
|      Data Scientist|       137-171 |   4.2|    Chantilly, VA|5001 to 10000 emp...|    Company - Public|Research & Develo...|   Business Services|       VA|         52|     0|    0|      0|
|      Data Scientist|       137-171 |   3.8|

In [9]:
df.printSchema()

root
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- job_state: string (nullable = true)
 |-- company_age: integer (nullable = true)
 |-- python: integer (nullable = true)
 |-- spark: integer (nullable = true)
 |-- tableau: integer (nullable = true)



In [10]:
#Split the data
df = df.withColumn("min_salary", split(col("Salary Estimate"), "-").getItem(0).cast("integer")).withColumn("max_salary", split(col("Salary Estimate"), "-").getItem(1).cast("integer"))

In [11]:
df.printSchema()

root
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- job_state: string (nullable = true)
 |-- company_age: integer (nullable = true)
 |-- python: integer (nullable = true)
 |-- spark: integer (nullable = true)
 |-- tableau: integer (nullable = true)
 |-- min_salary: integer (nullable = true)
 |-- max_salary: integer (nullable = true)



In [12]:
df.show()

+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+
|           Job Title|Salary Estimate|Rating|         Location|                Size|   Type of ownership|            Industry|              Sector|job_state|company_age|python|spark|tableau|min_salary|max_salary|
+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+
|   Sr Data Scientist|       137-171 |   3.1|     New York, NY|1001 to 5000 empl...|Nonprofit Organiz...|  Insurance Carriers|           Insurance|       NY|         27|     0|    0|      0|       137|       171|
|      Data Scientist|       137-171 |   4.2|    Chantilly, VA|5001 to 10000 emp...|    Company - Public|Research & Develo...|   Business Services| 

In [13]:
df = df.withColumn("average_salary", (col("min_salary") + col("max_salary") ) / 2)

In [15]:
df.printSchema()
df.show()

root
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- job_state: string (nullable = true)
 |-- company_age: integer (nullable = true)
 |-- python: integer (nullable = true)
 |-- spark: integer (nullable = true)
 |-- tableau: integer (nullable = true)
 |-- min_salary: integer (nullable = true)
 |-- max_salary: integer (nullable = true)
 |-- average_salary: double (nullable = true)

+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+--------------+
|           Job Title|Salary Estimate|Rating|         Location|                Size|   Type of ownershi

In [17]:
df = df.withColumn("Rating", when((col("Rating") == -1) | (col("Rating") == 0), 1).otherwise(col("Rating")))

In [18]:
df.show()

+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+--------------+
|           Job Title|Salary Estimate|Rating|         Location|                Size|   Type of ownership|            Industry|              Sector|job_state|company_age|python|spark|tableau|min_salary|max_salary|average_salary|
+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+--------------+
|   Sr Data Scientist|       137-171 |   3.1|     New York, NY|1001 to 5000 empl...|Nonprofit Organiz...|  Insurance Carriers|           Insurance|       NY|         27|     0|    0|      0|       137|       171|         154.0|
|      Data Scientist|       137-171 |   4.2|    Chantilly, VA|5001 to 10000 emp...|    

In [20]:
null_cnt = {c: df.filter(col(c).isNull()).count() for c in df.columns}

In [21]:
null_cnt

{'Job Title': 0,
 'Salary Estimate': 0,
 'Rating': 0,
 'Location': 0,
 'Size': 27,
 'Type of ownership': 27,
 'Industry': 71,
 'Sector': 71,
 'job_state': 0,
 'company_age': 0,
 'python': 0,
 'spark': 0,
 'tableau': 0,
 'min_salary': 0,
 'max_salary': 0,
 'average_salary': 0}

In [24]:
df = df.fillna(-1)

In [25]:
null_cnt = {c: df.filter(col(c).isNull()).count() for c in df.columns}
null_cnt

{'Job Title': 0,
 'Salary Estimate': 0,
 'Rating': 0,
 'Location': 0,
 'Size': 27,
 'Type of ownership': 27,
 'Industry': 71,
 'Sector': 71,
 'job_state': 0,
 'company_age': 0,
 'python': 0,
 'spark': 0,
 'tableau': 0,
 'min_salary': 0,
 'max_salary': 0,
 'average_salary': 0}

In [26]:
avg_sal = df.groupBy("Job Title").agg(mean("average_salary").alias("avg_salary"))

In [28]:
avg_sal.show()

+--------------------+-----------------+
|           Job Title|       avg_salary|
+--------------------+-----------------+
|Senior Data Scien...|99.33333333333333|
|Clinical Data Ana...|            164.5|
|Senior Business I...|             90.0|
|Data Analyst/Engi...|            115.5|
|Staff BI and Data...|            107.0|
|Intelligence Data...|            90.75|
|Report Writer-Dat...|             92.5|
|Hydrogen/Tritium ...|            148.0|
|Business Intellig...|           109.25|
|        Data Modeler|            154.0|
|Scientist / Group...|            197.5|
|Senior Research S...|            105.0|
|Software Engineer...|            164.5|
|   Sr Data Scientist|           126.75|
|COMPUTER SCIENTIS...|            271.5|
|Data Scientist/Ma...|            125.5|
|Data Scientist - ...|            120.5|
|  Decision Scientist|             94.5|
|Data Scientist - ...|            97.75|
|Data Scientist / ...|            128.5|
+--------------------+-----------------+
only showing top

In [29]:
df.show()

+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+--------------+
|           Job Title|Salary Estimate|Rating|         Location|                Size|   Type of ownership|            Industry|              Sector|job_state|company_age|python|spark|tableau|min_salary|max_salary|average_salary|
+--------------------+---------------+------+-----------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+-----+-------+----------+----------+--------------+
|   Sr Data Scientist|       137-171 |   3.1|     New York, NY|1001 to 5000 empl...|Nonprofit Organiz...|  Insurance Carriers|           Insurance|       NY|         27|     0|    0|      0|       137|       171|         154.0|
|      Data Scientist|       137-171 |   4.2|    Chantilly, VA|5001 to 10000 emp...|    

In [30]:
output_path = "Datasets/output/Jobs_data"

In [31]:
df.write.csv(output_path, header = True)